<a href="https://colab.research.google.com/github/AbigailBG153/Maching_Learning-Proyecto-/blob/main/Recoleccion_de_datos_TP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1019]:
import pandas as pd
import re

In [ ]:
import requests
import pandas as pd

# Definir la URL de la API para obtener la lista de criptomonedas
listings_url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical'

# Definir los parámetros de la solicitud para obtener la lista de criptomonedas
listings_parameters = {
  'date': '2024-03-27',  # Fecha de inicio de los datos históricos
  'convert': 'USD',             # Moneda en la que se convertirán los valores
  'cryptocurrency_type':'tokens',
  'limit': 1000                # Ajusta el límite para asegurarte de obtener todas las criptomonedas
}

# Definir los headers con la clave de API
headers = {
  'X-CMC_PRO_API_KEY': '83a12ab3-3579-4070-ab2f-e86210b78037'  # Reemplazar 'TU_CLAVE_DE_API' con tu clave de API
}

# Realizar la solicitud GET a la API para obtener la lista de criptomonedas
response = requests.get(listings_url, headers=headers, params=listings_parameters)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # La solicitud fue exitosa, obtener los datos de respuesta
    data = response.json()

    # Inicializar un DataFrame vacío para almacenar los datos
    data_token = pd.DataFrame()

    # Iterar sobre cada criptomoneda en la lista
    for cryptocurrency in data['data']:
        # Normalizar la data del token y guardar en un DataFrame
        df_tokens = pd.json_normalize(cryptocurrency)

        # Concatenar el DataFrame del token al DataFrame principal
        data_token = pd.concat([data_token, df_tokens], ignore_index=True)

    # Imprimir el DataFrame resultante
    print(data_token)
else:
    # La solicitud falló, imprimir el mensaje de error
    print("Error al obtener la lista de criptomonedas:", response.status_code, response.text)


Error al obtener la lista de criptomonedas: 400 {"status":{"timestamp":"2024-04-28T02:56:28.942Z","error_code":400,"error_message":"Your plan allows 1 months of historical access. Please upgrade your plan or choose a startDate that is newer than 2024-03-27T02:56:28.942Z.","elapsed":0,"credit_count":0,"notice":null}}


In [ ]:
data_token

In [ ]:
import os

# Crear el directorio 'data/raw' si no existe
raw_data_directory = 'data/raw'
os.makedirs(raw_data_directory, exist_ok=True)

# Definir el nombre del archivo CSV
csv_filename = f"data_tokens_{'2024-04-26'}.csv"  # Puedes cambiar '2024-03-01' por la fecha real si lo deseas

# Ruta completa del archivo CSV
csv_filepath = os.path.join(raw_data_directory, csv_filename)

# Guardar el DataFrame como un archivo CSV
data_token.to_csv(csv_filepath, index=False)

# Confirmación de guardado
print(f"Los datos se han guardado correctamente en: {csv_filepath}")


Los datos se han guardado correctamente en: data/raw/data_tokens_2024-04-26.csv


In [1020]:
dia = '2024-04-26'

In [1021]:
data_1 = pd.read_csv(f'/content/data/raw/data_tokens_{dia}.csv')

In [ ]:
data_1.info() #id,name,symbol,tags,quote.USD.price,1000 non-null   float64

In [1023]:
data_1_selected = data_1[['id', 'name', 'symbol', 'tags', 'quote.USD.price']]
specific_date = pd.to_datetime('2024-04-27')
data_1_selected.insert(0, 'date', specific_date)

In [ ]:
data_1_selected['IA'] = 0
data_1_selected['Gaming'] = 0
data_1_selected['RWA'] = 0
data_1_selected['Meme'] = 0

In [1025]:
keywords = {
    'IA': ['AI & Big Data'],
    'Gaming': ['Gaming'],
    'RWA': ['Real World Assets'],
    'Meme': ['Meme']
}

In [1026]:
# Diccionario que mapea palabras clave a sus respectivos números
keyword_mapping = {'ai-big-data': 1,'gaming': 2, 'real-world-assets': 3,'memes': 4,}

In [1027]:
# Iterar sobre las filas del DataFrame
for index, row in data_1_selected.iterrows():
    tags_str = row['tags']  # Obtener la cadena de texto en la celda de la columna "tags"
    tags_list = eval(tags_str)  # Convertir la cadena de texto en una lista

    found_keywords = set()  # Conjunto para registrar qué palabras clave se encuentran

    # Buscar las palabras clave y agregarlas al conjunto
    for keyword in tags_list:
        if keyword in keyword_mapping:
            found_keywords.add(keyword_mapping[keyword])

    # Asignar los valores correspondientes a las columnas "IA", "Gaming", "RWA" y "Meme"
    for col_name, keyword_num in {'IA': 1, 'Gaming': 2, 'RWA': 3, 'Meme': 4}.items():
        data_1_selected.at[index, col_name] = 1 if keyword_num in found_keywords else 0

In [ ]:
data_1_selected

In [1029]:

# Lista para almacenar los portfolios encontrados únicos
unique_portfolios = []

# Iterar sobre las filas del DataFrame
for index, row in data_1_selected.iterrows():
    tags_str = row['tags']  # Obtener la cadena de texto en la celda de la columna "tags"
    tags_list = eval(tags_str)  # Convertir la cadena de texto en una lista

    # Buscar palabras que contengan "portfolio" en la lista de tags
    for tag in tags_list:
        if re.search(r'\b\w+-portfolio\b', tag):
            # Agregar el portfolio encontrado a la lista si no está presente
            if tag not in unique_portfolios:
                unique_portfolios.append(tag)

# Imprimir la lista de portfolios encontrados únicos
print(unique_portfolios)

['coinbase-ventures-portfolio', 'framework-ventures-portfolio', 'polychain-capital-portfolio', 'electric-capital-portfolio', 'fenbushi-capital-portfolio', 'hashkey-capital-portfolio', 'a16z-portfolio', 'multicoin-capital-portfolio', 'exnetwork-capital-portfolio', 'kenetic-capital-portfolio', 'alameda-research-portfolio', 'three-arrows-capital-portfolio', 'blockchain-capital-portfolio', 'defiance-capital-portfolio', 'pantera-capital-portfolio', 'paradigm-portfolio', 'dwf-labs-portfolio', 'arrington-xrp-capital-portfolio', 'dcg-portfolio', 'egirl-capital-portfolio', 'dragonfly-capital-portfolio', '1confirmation-portfolio', 'placeholder-ventures-portfolio', 'standard-crypto-portfolio', 'coinfund-portfolio', 'fabric-ventures-portfolio', 'ledgerprime-portfolio', 'binance-labs-portfolio', 'cms-holdings-portfolio', 'okx-ventures-portfolio', 'animoca-brands-portfolio', 'boostvc-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'winklevoss-capital-portfolio', 'vbc-ventures-port

In [1030]:
# DataFrame nuevo para almacenar la información
nueva_data = pd.DataFrame(columns=['date', 'portfolio', 'nombre_token', 'symbol_token','IA','Gaming',
                                   'RWA','Meme'])


In [1031]:
# Iterar sobre la lista de portfolios únicos
for portfolio in unique_portfolios:
    portfolio_found = False
    # Iterar sobre las filas del DataFrame
    for index, row in data_1_selected.iterrows():
        tags_str = row['tags']  # Obtener la cadena de texto en la celda de la columna "tags"
        tags_list = eval(tags_str)  # Convertir la cadena de texto en una lista

        # Buscar si el portfolio está presente en la lista de tags
        if portfolio in tags_list:
            # Agregar la información al DataFrame nuevo
            nueva_data.loc[len(nueva_data)] =[row['date'],
                                             portfolio,
                                            row['name'],
                                            row['symbol'],
                                            row['IA'],
                                            row['Gaming'],
                                            row['RWA'],
                                            row['Meme']]
            portfolio_found = True
            # Terminar de analizar este portfolio
            break
     # Si no se encuentra el portfolio en ninguna fila, agregar una fila con valores nulos
    if not portfolio_found:
        nueva_data.loc[len(nueva_data)] = [None, portfolio, None, None]

In [1032]:
# Crear el directorio 'data/raw' si no existe
raw_data_directory = 'data/portafolios'
os.makedirs(raw_data_directory, exist_ok=True)

# Definir el nombre del archivo CSV
csv_filename = f"data_portafolios_{dia}.csv"  # Puedes cambiar '2024-03-01' por la fecha real si lo deseas

# Ruta completa del archivo CSV
csv_filepath = os.path.join(raw_data_directory, csv_filename)

# Guardar el DataFrame como un archivo CSV
nueva_data.to_csv(csv_filepath, index=False)

# Confirmación de guardado
print(f"Los datos se han guardado correctamente en: {csv_filepath}")


Los datos se han guardado correctamente en: data/portafolios/data_portafolios_2024-04-26.csv


In [1033]:
import os
import pandas as pd
import glob

# Directorio donde se encuentran los archivos CSV
raw_data_directory = 'data/portafolios'

# Obtener la lista de todos los archivos CSV en el directorio
csv_files = glob.glob(os.path.join(raw_data_directory, '*.csv'))

# Lista para almacenar los DataFrames de los archivos CSV
data_frames = []

# Iterar sobre los archivos CSV y leer cada uno como un DataFrame
for csv_file in csv_files:
    data_frames.append(pd.read_csv(csv_file))

# Concatenar todos los DataFrames en uno solo
data_2 = pd.concat(data_frames, ignore_index=True)


In [1034]:
data_2

,date,portfolio,nombre_token,symbol_token,IA,Gaming,RWA,Meme
0,2024-04-27,coinbase-ventures-portfolio,USDC,USDC,0,0,0,0
1,2024-04-27,framework-ventures-portfolio,Chainlink,LINK,0,0,1,0
2,2024-04-27,polychain-capital-portfolio,Internet Computer,ICP,0,0,1,0
3,2024-04-27,electric-capital-portfolio,Internet Computer,ICP,0,0,1,0
4,2024-04-27,fenbushi-capital-portfolio,Internet Computer,ICP,0,0,1,0
...,...,...,...,...,...,...,...,...
1328,2024-04-27,polygon-ventures-portfolio,CyberConnect,CYBER,0,0,0,0
1329,2024-04-27,usv-portfolio,Numeraire,NMR,1,0,0,0
1330,2024-04-27,circle-ventures-portfolio,Zebec,ZBC,0,0,0,0
1331,2024-04-27,crypto-com-capital-portfolio,Artificial Liquid Intelligence,ALI,1,0,0,0


In [1036]:
# Crear el directorio 'data/raw' si no existe
raw_data_directory = 'data'
os.makedirs(raw_data_directory, exist_ok=True)

# Definir el nombre del archivo CSV
csv_filename = f"Dataset-portafolio-detallado.csv"  # Puedes cambiar '2024-03-01' por la fecha real si lo deseas

# Ruta completa del archivo CSV
csv_filepath = os.path.join(raw_data_directory, csv_filename)

# Guardar el DataFrame como un archivo CSV
data_2.to_csv(csv_filepath, index=False)

# Confirmación de guardado
print(f"Los datos se han guardado correctamente en: {csv_filepath}")


Los datos se han guardado correctamente en: data/Dataset-portafolio-detallado.csv


In [1037]:
from google.colab import files
import shutil

# Selecciona el archivo CSV desde tu sistema local
uploaded = files.upload()
# Ruta del archivo CSV cargado en Colab
csv_file_path = '/content/Dataset-portafolio.csv'  # Cambia 'nombre_del_archivo.csv' al nombre real de tu archivo
# Ruta de la carpeta de destino
destination_folder = 'data'  # Cambia 'carpeta_de_destino' al nombre de tu carpeta de destino

# Mover el archivo CSV a la carpeta de destino
shutil.move(csv_file_path, destination_folder)


Saving Dataset-portafolio.csv to Dataset-portafolio (1).csv


'data/Dataset-portafolio.csv'

In [1038]:
shutil.rmtree('/content/data/portafolios(1)')

In [1043]:
import os
import shutil
import zipfile

# Carpeta que deseas comprimir
folder_to_compress = '/content/data'  # Cambia 'carpeta_a_comprimir' al nombre de la carpeta que deseas comprimir

# Comprimir la carpeta en un archivo RAR
output_zip_filename = 'Dataset.zip'
with zipfile.ZipFile(output_zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_to_compress):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_compress))

# Descargar el archivo ZIP
from google.colab import files
files.download(output_zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>